<a href="https://colab.research.google.com/github/zanqi/tinybert/blob/main/bert_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Getting Data

In [1]:
# We always start with a dataset to train on. Let's download the MIND dataset
!wget https://raw.githubusercontent.com/zanqi/tinybert/main/news.tsv.zip


--2023-07-13 01:45:48--  https://raw.githubusercontent.com/zanqi/tinybert/main/news.tsv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10694395 (10M) [application/zip]
Saving to: ‘news.tsv.zip’

news.tsv.zip        100%[===================>]  10.20M  --.-KB/s    in 0.09s   

2023-07-13 01:45:48 (115 MB/s) - ‘news.tsv.zip’ saved [10694395/10694395]



In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O shakespeare.txt

zsh:1: command not found: wget


In [1]:
! unzip {'news.tsv.zip'}

Archive:  news.tsv.zip
  inflating: news.tsv                


In [9]:
! pip3 install transformers nltk pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.9 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 502 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 2.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zanqiliang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
import pandas as pd
import nltk
from transformers import BertTokenizer

with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

sents = nltk.sent_tokenize(text) # list(str)

n = len(sents)
train_data = sents[:int(n*0.9)]
val_data = sents[int(n*0.9):]

#shakespeare
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data), (batch_size,))
    sents1 = [data[i] for i in ix]
    x = tokenizer(sents1, return_tensors="pt", padding=True, truncation=True)

    token_ids = torch.LongTensor(x["input_ids"])
    attention_mask = torch.LongTensor(x["attention_mask"])
    special = torch.isin(token_ids, torch.tensor(tokenizer.all_special_ids))
    mask = torch.rand(token_ids.shape) < 0.15
    
    mask = mask & (attention_mask == 1) & ~special
    mask_mask = (torch.rand(token_ids.shape) < 0.8) & mask
    mask_rand = (torch.rand(token_ids.shape) < 0.5) & mask & ~mask_mask

    y = token_ids.detach().clone()
    y[~mask] = -100
    counts.update(y[y != -100].tolist())

    token_ids[mask_mask] = tokenizer.mask_token_id
    token_ids[mask_rand] = torch.randint(
        tokenizer.vocab_size, token_ids[mask_rand].shape
    )

    return x.to(device), y.to(device), None

/Users/zanqiliang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/zanqiliang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
data = pd.read_csv('news.tsv',header=None,sep='\t')

data.columns=['News ID',
"Category",
"SubCategory",
"Title",
"Abstract",
"URL",
"Title Entities",
"Abstract Entities "]

text_data = data.iloc[:, 3:5]
text_data.Abstract.str.len().describe()

# abstracts = text_data.Abstract
sents_by_abstract = [nltk.sent_tokenize(a) for a in text_data[text_data.Abstract.str.len() > 10].Abstract]
abstracts = [sents for sents in sents_by_abstract if len(sents) > 1]
# len(abstracts), abstracts[:10]

zipped = [zip(a, a[1:]) for a in abstracts]
pairs = [(s1, s2) for z in zipped for s1, s2 in z]

n = int(0.9*len(pairs)) # first 90% will be train, rest val
train_data = pairs[:n]
val_data = pairs[n:]

# news
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data), (batch_size,))
    ix_mask = torch.rand(batch_size) < 0.5
    next_ix = ix.detach().clone()
    next_ix[ix_mask] = (ix[ix_mask] + 10) % len(data)
    sents1 = ["".join(data[i]) for i in ix]
    # sents1 = ["".join(data[i][: len(data[i]) // 2]) for i in ix]
    # sents2 = ["".join(data[i][len(data[i]) // 2 :]) for i in next_ix]
    # x = tokenizer(sents1, sents2, return_tensors="pt", padding=True, truncation=True)
    x = tokenizer(sents1, return_tensors="pt", padding=True, truncation=True)

    token_ids = torch.LongTensor(x["input_ids"])
    attention_mask = torch.LongTensor(x["attention_mask"])
    special = torch.isin(token_ids, torch.tensor(tokenizer.all_special_ids))
    mask = torch.rand(token_ids.shape) < 0.15
    most_common = torch.isin(
        token_ids, torch.tensor(counts.most_common(len(counts) // 20))
    )
    mask = mask & (attention_mask == 1) & ~special & ~most_common
    # mask = mask & (attention_mask == 1) & ~special
    mask_mask = (torch.rand(token_ids.shape) < 0.8) & mask
    mask_rand = (torch.rand(token_ids.shape) < 0.5) & mask & ~mask_mask

    y = token_ids.detach().clone()
    y[~mask] = -100
    counts.update(y[y != -100].tolist())

    token_ids[mask_mask] = tokenizer.mask_token_id
    token_ids[mask_rand] = torch.randint(
        tokenizer.vocab_size, token_ids[mask_rand].shape
    )

    nsp_target = ix == next_ix
    return x.to(device), y.to(device), nsp_target.long().to(device)

In [5]:
text_data.iloc[text_data.Abstract.str.len().argmax()].Abstract

'Real talk. Demi Moore got candid about a variety of topics in her new book, Inside Out, including her famous exes, substance abuse struggles and her heartbreaking sexual assault. "The same question kept going through my head: How did I get here?" the 56-year-old actress began in the memoir, which was released on Tuesday, September 24. "The husband who I\'d thought was the love of my life had cheated on me and then decided he didn\'t want to work on our marriage. My children weren\'t speaking me. … Is this life? I wondered. Because if this is it, I\'m done." Moore provided insight into all three of her marriages in the book. She was married to Freddy Moore from 1980 to 1985, Bruce Willis from 1987 to 2000 and Ashton Kutcher from 2005 to 2013. The end of the G.I. Jane star\'s relationship with the former That 70\'s Show star, however, seemed to have the biggest impact on her. "I lost me," the Ghost actress told Diane Sawyer on Good Morning America on Monday, September 23, about their sp

In [6]:
text_data[text_data.Abstract.str.len() > 10]

,Title,Abstract
0,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
...,...,...
51275,Realme takes chunk of India mobile market as S...,Over 400 percent more phones shipped year-on-year
51276,Young Northeast Florida fans flock to U.S. wom...,When the U.S. women's national soccer team arr...
51277,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...
51279,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b..."


In [8]:
sent_text = nltk.sent_tokenize('My name is John E. Doe. Please turn to p. 55.')
sent_text

['My name is John E. Doe.', 'Please turn to p. 55.']

In [11]:
print("length of dataset in sentences: ", sum(len(a) for a in abstracts))
print("length of dataset in words: ", sum(len(nltk.word_tokenize(s)) for a in abstracts for s in a))

length of dataset in sentences:  74948
length of dataset in words:  1427558


In [15]:
print(train_data[0])
encoded = tokenizer(train_data[1])
print(encoded) # len(token_ids), tokenizer.decode(token_ids)
tokenizer.batch_decode(encoded['input_ids'])


('Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine.', 'Next to him was an empty helmet propped up to trick snipers, already perforated with multiple holes.')
{'input_ids': [[101, 1045, 2371, 2066, 1045, 2001, 1037, 9861, 1010, 1998, 2108, 2019, 6452, 2564, 2134, 1005, 1056, 2393, 2008, 1012, 102], [101, 1999, 2755, 1010, 2009, 3053, 3908, 2033, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


["[CLS] i felt like i was a fraud, and being an nba wife didn't help that. [SEP]",
 '[CLS] in fact, it nearly destroyed me. [SEP]']

In [16]:
import random

import torch
# random.seed(1337)

random.random()
ix = torch.randint(100, (4,))
mask = torch.rand(4) < 0.5
next_ix = ix.detach().clone()
next_ix[mask] = (ix[mask] + 10) % 100
ix, next_ix

(tensor([54, 23, 93, 26]), tensor([64, 33,  3, 36]))

In [17]:
import torch
import random
random.seed(1337)
torch.manual_seed(1337)

batch_size = 4

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data), (batch_size,))
  ix_mask = torch.rand(batch_size) < 0.5
  next_ix = ix.detach().clone()
  next_ix[ix_mask] = (ix[ix_mask] + 10) % len(data)
  sents1 = [data[i][0] for i in ix]
  sents2 = [data[i][1] for i in next_ix]
  x = tokenizer(sents1, sents2, return_tensors="pt", padding=True, truncation=True)

  token_ids = torch.LongTensor(x["input_ids"])
  attention_mask = torch.LongTensor(x["attention_mask"])
  cls = token_ids == tokenizer.cls_token_id
  sep = token_ids == tokenizer.sep_token_id
  special = cls | sep
  mask = torch.rand(token_ids.shape) < 0.15
  mask = mask & (attention_mask == 1) & ~special
  mask_mask = (torch.rand(token_ids.shape) < 0.8) & mask
  mask_rand = (torch.rand(token_ids.shape) < 0.5) & mask & ~mask_mask

  y = token_ids.detach().clone()
  y[~mask] = -100

  token_ids[mask_mask] = tokenizer.mask_token_id
  token_ids[mask_rand] = torch.randint(
      0, tokenizer.vocab_size, token_ids[mask_rand].shape
  )

  nsp_target = ix == next_ix
  return x, y, nsp_target.float().unsqueeze(-1)

In [18]:
xb, yb, nsp_targets = get_batch('train')
tokenizer.decode(xb['input_ids'][0]), tokenizer.decode(yb[0]), yb.shape
print(nsp_targets)

input = [[tokenizer.decode([id]) for id in x] for x in xb['input_ids']]
target = [[tokenizer.decode([id]) for id in x] for x in yb]


import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
df = pd.DataFrame({"input": input[0], "target": target[0]})
df.T
# for t1, t2 in zip(xb['input_ids'][0], yb[0]):
#   if t2 == -1:
#     print(tokenizer.decode([t1]))
#   else:
#     print(f'{tokenizer.decode([t1])} => {tokenizer.decode([t2])}')





tensor([[0.],
        [0.],
        [1.],
        [1.]])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70
input,[CLS],moore,",",sc,[MASK],fox,carolina,),-,spartan,##burg,county,deputies,say,a,woman,[MASK],recovering,after,being,shot,several,[MASK],by,her,boyfriend,sunday,afternoon,.,[SEP],construction,on,the,$,[MASK],million,project,[MASK],now,expected,to,begin,early,[MASK],spring,-,-,about,[MASK],months,connectivity,than,first,[MASK],[MASK],-,with,o,[MASK],##up,##ancy,of,[MASK],##s,by,the,end,.,.,.,[SEP]
target,[UNK],[UNK],[UNK],[UNK],(,[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],is,[UNK],[UNK],[UNK],[UNK],[UNK],times,[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],26,[UNK],[UNK],is,[UNK],[UNK],[UNK],[UNK],[UNK],next,[UNK],[UNK],[UNK],[UNK],six,[UNK],later,[UNK],[UNK],forecast,-,[UNK],[UNK],[UNK],##cc,[UNK],[UNK],[UNK],rental,[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK],[UNK]


### Simple Model

In [19]:
import torch.nn as nn
from torch.nn import functional as F

class SimpleModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.word_embedding_table = nn.Embedding(vocab_size, 64)
    self.lm_head = nn.Linear(64, vocab_size, bias=False)
    self.lm_head.weight = self.word_embedding_table.weight
    self.mask_token_id = tokenizer.mask_token_id

  def forward(self, x, targets=None):
    embedding = self.word_embedding_table(x['input_ids']) # (B, T, C)
    logits = self.lm_head(embedding)

    if targets is None:
      return logits
    else:
      return logits, F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-100)

  def predict(self, x):
    idx = x['input_ids'] # (B, T)
    x['input_ids'] = torch.concat((torch.zeros(idx.shape[0], 1), idx[:, :-1]), dim=1).long()
    logits = self(x) # (B, T, C)
    mask = idx == self.mask_token_id # (B, T)
    probs = F.softmax(logits[mask], dim=-1) # (T_masked, C)
    # idx_masked = torch.multinomial(probs, num_samples=1) # (B, T_masked)
    top_probs = torch.topk(probs, 3, dim=-1) # (T_masked, 3)
    res = []
    for i in range(3):
      idx = idx.detach().clone() # (B, T)
      idx[mask] = top_probs.indices[:, i].view(-1)
      res.append(idx)
    res = torch.stack(res)
    res = res.permute(1, 0, 2)

    return res # (3, B, T)
    #  want (B, 3, T)

model = SimpleModel(tokenizer.vocab_size)

In [20]:
logits, loss = model(xb, yb)
print(logits.shape, loss.shape)
print(loss)


torch.Size([4, 71, 30522]) torch.Size([])
tensor(70.4512, grad_fn=<NllLossBackward0>)


In [21]:
x = ["I'm Afraid to Tell [MASK] Male Bosses I'm [MASK]",
     "The truth is that Tiger has been a nicer [MASK] than he has received credit for."]
x = tokenizer(x, return_tensors="pt", padding=True, truncation=True)
pred = model.predict(x)
pred.shape, tokenizer.batch_decode(pred[0]), tokenizer.batch_decode(pred[1])

(torch.Size([2, 3, 20]),
 ["[CLS] i'm afraid to tell tell male bosses i'm m [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]",
  "[CLS] i'm afraid to tell fleet male bosses i'm crack [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]",
  "[CLS] i'm afraid to tell hilary male bosses i'm bryn [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]"],
 ['[CLS] the truth is that tiger has been a nicerr than he has received credit for. [SEP]',
  '[CLS] the truth is that tiger has been a nicer dreamed than he has received credit for. [SEP]',
  '[CLS] the truth is that tiger has been a nicer regaining than he has received credit for. [SEP]'])

In [22]:
x = ["I'm Afraid to Tell [MASK] Male Bosses I'm [MASK]", "I'm Afraid to Tell [MASK] Male Bosses I'm [MASK]"]
x = tokenizer(x, return_tensors="pt", padding=True, truncation=True)
idx = x['input_ids']
mask = idx == tokenizer.mask_token_id
# print(mask.shape)
logits = torch.randn(2, 15, 4)
# print(logits[mask].shape)
probs = F.softmax(logits[mask], dim=-1) # (T_masked, C)
top_probs = torch.topk(probs, 3, dim=-1) #(T_masked, C)
# print(idx[mask].shape)
print(top_probs.indices)
idx[mask] = top_probs.indices[:, 0].view(-1)
print(top_probs.indices[:, 0].view(-1))
idx
# idx[mask].shape
# pred = model.predict(x)
# tokenizer.batch_decode(pred)

tensor([[2, 1, 0],
        [1, 3, 2],
        [2, 3, 1],
        [3, 2, 1]])
tensor([2, 1, 2, 3])


tensor([[  101,  1045,  1005,  1049,  4452,  2000,  2425,     2,  3287, 23029,
          1045,  1005,  1049,     1,   102],
        [  101,  1045,  1005,  1049,  4452,  2000,  2425,     2,  3287, 23029,
          1045,  1005,  1049,     3,   102]])

### Attention

In [23]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

logits = torch.randn((B, T, T))
attn = F.softmax(logits, dim=-1)
print(attn.sum(dim=-1)) # probs sum to 1 for each token/row
res = attn @ x
attn.shape, res.shape

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])


(torch.Size([4, 8, 8]), torch.Size([4, 8, 32]))

In [24]:
# for self-attention, logits come from x
logits = x @ x.transpose(-1, -2) # (B, T, C) @ (B, C, T) ---> (B, T, T)

logits2 = []
for i in range(B):
  logits_b = []
  for j in range(T):
    logits_b.append(x[i, j] @ x[i].T) # (, C) @ (C, T) ---> (, T)

  logits2.append(torch.stack(logits_b)) # (T, T)
logits2 = torch.stack(logits2)

torch.allclose(logits, logits2)

True

In [25]:
import torch.nn as nn

key = nn.Linear(C, C, bias=False)
query = nn.Linear(C, C, bias=False)
value = nn.Linear(C, C, bias=False)
q = query(x)
k = key(x)
v = value(x)

logits = q @ k.transpose(-1, -2) # (B, T, C) @ (B, C, T) ---> (B, T, T)
attn = F.softmax(logits, dim=-1)
res = attn @ v # (B, T, T) @ (B, T, C)
res.shape

torch.Size([4, 8, 32])

Multi-head

In [26]:
H = 2
C_H = C // H
q_h = q.view(B, T, H, -1).transpose(1, 2)
k_h = k.view(B, T, H, -1).transpose(1, 2)
# logits = torch.randn((B, H, T, T))
logits = q_h @ k_h.transpose(-1, -2) # (B, H, T, C_H) @ (B, H, C_H, T) ---> (B, H, T, T)
attn = F.softmax(logits, dim=-1)
v_h = v.view(B, T, H, -1).transpose(1, 2)
res = attn @ v_h # (B, H, T, T) @ (B, H, T, C/H) ---> (B, H, T, C/H)
res.shape
res.transpose(1, 2).reshape(B, T, -1).shape

torch.Size([4, 8, 32])

Scaled: formular Var(aX), Var(X+Y) and Var(XY) in https://en.wikipedia.org/wiki/Variance

In [27]:
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1))

# Increasing the variance of the logits makes the probs moving to more peaky,
# coverging to one hot

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [28]:
k = torch.randn(100)
q = torch.randn(100)
prod = k * q
print(k.var(), q.var(), prod.var()) # var(XY) = 1

k = torch.randn(T, 100)
q = torch.randn(T, 100)
logits = q @ k.T
logits.var() # var(X+Y) = 2*var(X)

tensor(0.9592) tensor(0.9445) tensor(0.8309)


tensor(93.2930)

In [29]:
k = torch.randn(B,T,C_H)
q = torch.randn(B,T,C_H)
# var(k @ q.T) = C_H * var(k)
# var(k @ q.T * x) = C_H * var(k) * x^2
# x^2 = 1/C_H
wei = q @ k.transpose(-2, -1) * C_H**-0.5
wei.var()

tensor(1.2408)

### Bert Model

In [30]:
# BertModel
#   word_embedding, pos_emdding, tk_type_emdding
#   BertLayer *
#     SelfAttention
#     mlp
#   cls_head
#   lm_head

In [31]:
a = torch.tensor([1,2,3])
torch.isin(a, torch.tensor([1]))
tokenizer.all_special_ids
len(tokenizer)

30522

In [13]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer

hidden_size = 128
num_head = 4
drop_out = 0.0
num_layers = 12
device = "cuda" if torch.cuda.is_available() else "mps"
max_iters = 60_000
eval_interval = 100
lr = 1e-4
eval_iters = 200
batch_size = 16


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
counts = Counter()

class SelfAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.head_size = hidden_size // num_head

        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)

        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(drop_out)

    def forward(self, x, mask):
        B, T, C = x.shape
        k = self.key(x)
        k = k.view(B, T, num_head, -1).transpose(1, 2)  # (B, num_head, T, head_size)
        v = self.value(x)
        v = v.view(B, T, num_head, -1).transpose(1, 2)  # (B, num_head, T, head_size)
        q = self.query(x)
        q = q.view(B, T, num_head, -1).transpose(1, 2)  # (B, num_head, T, head_size)

        logits = q @ k.transpose(-1, -2) / self.head_size**0.5
        mask = mask[:, None, None, :]  # (bs, 1, 1, seq_len)
        logits = logits.masked_fill(mask == 0, float("-inf"))
        probs = F.softmax(logits, dim=-1)
        probs = self.dropout(probs)
        res = probs @ v  # (B, num_head, T, head_size)
        res = res.transpose(1, 2).reshape(B, T, -1)  # (B, T, C)
        return self.dropout(self.dense(res))  # (B, T, C)


class BertLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = SelfAttention()
        self.ffwd = nn.Sequential(
            nn.Linear(hidden_size, 4 * hidden_size),
            nn.ReLU(),
            nn.Linear(4 * hidden_size, hidden_size),
            nn.Dropout(drop_out),
        )
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)

    def forward(self, x, mask):
        x = x + self.sa(self.ln1(x), mask)
        x = x + self.ffwd(self.ln2(x))
        return x


class BertModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.word_embedding = nn.Embedding(
            tokenizer.vocab_size, hidden_size, padding_idx=tokenizer.pad_token_id
        )
        self.pos_embedding = nn.Embedding(tokenizer.model_max_length, hidden_size)
        self.tk_type_embedding = nn.Embedding(2, hidden_size)
        position_ids = torch.arange(tokenizer.model_max_length).unsqueeze(0)
        self.register_buffer("position_ids", position_ids)
        self.dropout = nn.Dropout(drop_out)
        self.ln = nn.LayerNorm(hidden_size)

        self.bert_layers = nn.ModuleList([BertLayer() for _ in range(num_layers)])
        self.ln_final = nn.LayerNorm(hidden_size)

        self.cls_dense = nn.Linear(hidden_size, hidden_size)
        self.cls_af = nn.Tanh()
        self.cls_head = nn.Linear(hidden_size, 2)

        self.lm_ln = nn.LayerNorm(hidden_size)
        self.lm_head = nn.Linear(hidden_size, tokenizer.vocab_size, bias=False)
        self.lm_head.weight = self.word_embedding.weight

    def forward(self, encoding, nsp_targets=None, lm_targets=None):
        ids = encoding["input_ids"]  # (B, T)
        type_ids = encoding["token_type_ids"]
        atten_mask = encoding["attention_mask"]

        B, T = ids.shape
        word_embedding = self.word_embedding(ids)
        pos_ids = self.position_ids[:, :T]
        pos_embedding = self.pos_embedding(pos_ids)
        tk_type_embedding = self.tk_type_embedding(type_ids)
        embedding = word_embedding + pos_embedding + tk_type_embedding
        embedding = self.dropout(self.ln(embedding))

        for bert_layer in self.bert_layers:
            embedding = bert_layer(embedding, atten_mask)
        embedding = self.ln_final(embedding)

        cls_logit = self.cls_dense(embedding[:, 0])
        cls_logit = self.cls_head(self.cls_af(cls_logit))
        lm_logits = self.lm_head(embedding)

        cls_loss = None
        if nsp_targets is not None:
            cls_loss = F.cross_entropy(cls_logit, nsp_targets)

        lm_loss = None
        if lm_targets is not None:
            lm_loss = F.cross_entropy(
                lm_logits.view(-1, lm_logits.size(-1)),
                lm_targets.view(-1),
                ignore_index=-100,
            )

        return cls_logit, lm_logits, cls_loss, lm_loss

    def predict(self, encoding):
        cls_logit, lm_logits, _, _ = self(encoding)  # (B, 1) (B, T, C)
        probs = F.softmax(lm_logits, dim=-1)  # (B, T, C)
        top_probs = torch.topk(probs, 3, dim=-1)  # (B, T, 3)

        nsp = F.softmax(cls_logit, dim=-1)

        #  (B, 3, T), (B, 2)
        return top_probs.indices.transpose(1, 2), nsp


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        nsp_losses = torch.zeros(eval_iters)
        lm_losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y, nsp_targets = get_batch(split)
            _, _, nsp_loss, lm_loss = model(X, nsp_targets=nsp_targets, lm_targets=Y)
            nsp_losses[k] = nsp_loss.item() if nsp_loss is not None else 0
            lm_losses[k] = lm_loss.item() if lm_loss is not None else 0
        out[split] = nsp_losses.mean(), lm_losses.mean()
    model.train()
    return out


model = BertModel()
model = model.to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")
# for n, p in model.named_parameters():
#   print(n, p.numel()/1e6, 'M')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)


def train():
    for iter in range(max_iters):
        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(
                f"step {iter}: train loss nsp={losses['train'][0]:.4f} lm={losses['train'][1]:.4f}, val loss nsp={losses['val'][0]:.4f} lm={losses['val'][1]:.4f}"
            )

        # sample a batch of data
        xb, yb, nsp_targets = get_batch("train")

        # evaluate the loss
        _, _, nsp_loss, lm_loss = model(xb, lm_targets=yb)
        optimizer.zero_grad(set_to_none=True)
        # loss = nsp_loss + lm_loss
        lm_loss.backward()
        optimizer.step()


train()

6.36941 M parameters
step 0: train loss nsp=0.7088 lm=2.4274, val loss nsp=0.7062 lm=1.9838


KeyboardInterrupt: 

In [3]:
tokenizer.decode([token for token, count in counts.most_common(20)]), counts

(", :. the'and i to ; of a you my? that! in is not -",
 Counter({1010: 427787,
          1024: 233371,
          1012: 176537,
          1996: 131037,
          1005: 130416,
          1998: 124645,
          1045: 112082,
          2000: 102221,
          1025: 78459,
          1997: 74980,
          1037: 72408,
          2017: 71479,
          2026: 65548,
          1029: 55197,
          2008: 53230,
          999: 49892,
          1999: 49558,
          2003: 46694,
          2025: 43537,
          1011: 42898,
          2005: 41761,
          2022: 39265,
          2007: 38846,
          2033: 38815,
          2009: 38726,
          1055: 38329,
          2002: 34264,
          15223: 33608,
          2115: 32862,
          2021: 31861,
          2023: 31217,
          2031: 30556,
          2004: 29973,
          2010: 29270,
          1040: 29187,
          2054: 27695,
          2061: 25789,
          2015: 25572,
          15177: 23755,
          2032: 23536,
          2080: 

In [9]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

xb, yb, nsp_targets = get_batch('train')
input = [[tokenizer.decode([id]) for id in x] for x in xb['input_ids']]
target = [[tokenizer.decode([id]) if id != -100 else "" for id in y] for y in yb]
# model(xb, lm_targets=yb)
pred, nsp = model.predict(xb)

print(xb['input_ids'].shape, yb.shape, pred.shape)
# print(xb['input_ids'][0])
# print(yb[0])
# print(xb['token_type_ids'][0])
# print(xb['attention_mask'][0])

pred_list = [[[tokenizer.decode([id]) if yid != -100 else "" for id, yid in zip(p, y)] for p in p3] for p3, y in zip(pred, yb)]
# len(input), len(target), len(pred_list)
df = pd.DataFrame({'input': input[0], 'target': target[0],
                   'pred1': pred_list[0][0], # (B, rank)
                   'pred2': pred_list[0][1],
                   'pred3': pred_list[0][2]})
df.T


torch.Size([16, 83]) torch.Size([16, 83]) torch.Size([16, 3, 83])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82
input,[CLS],first,[MASK],##sp,[MASK],##tor,:,so,he,did,",",my,lord,:,the,[MASK],marvel,##l,',d,at,[MASK],",",and,",",in,##ʑ,last,",",when,[MASK],had,carried,rome,and,[MASK],we,look,',[MASK],for,no,less,spoil,than,glory,[MASK],-,-,[MASK],##idi,##us,[MASK],there,was,it,:,for,which,my,sin,##ew,##s,[MASK],be,[MASK],',d,[MASK],him,.,[SEP],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD]
target,,,con,,##ira,,,,,,,,,,,army,,,,,,it,,,,,the,last,,,he,,,,,that,,,,d,,,,,,,",",,,auf,,,:,,,,,,,,,,,shall,,stretch,,,upon,,,,,,,,,,,,,,
pred1,,,con,,##ira,,,,,,,,,,,",",,,,,,",",,,,,",",last,,,",",,,,,",",,,,",",,,,,,,",",,,",",,,",",,,,,,,,,,,",",,",",,,",",,,,,,,,,,,,,,
pred2,,,king,,",",,,,,,,,,,,:,,,,,,the,,,,,s,moved,,,he,,,,,and,,,,;,,,,,,,he,,,and,,,:,,,,,,,,,,,to,,to,,,and,,,,,,,,,,,,,,
pred3,,,if,,##man,,,,,,,,,,,;,,,,,,##s,,,,,;,had,,,i,,,,,:,,,,:,,,,,,,-,,,a,,,and,,,,,,,,,,,and,,the,,,the,,,,,,,,,,,,,,


In [35]:
xb, yb, nsp = get_batch('train')
yb

tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, 2698,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]])